# Adversarial attacks

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

from adv_utils import (
    download_file,
    FGSMAttack,
    PGDAttack
)

## Load model

In [ ]:
# set model weights
weights = ResNet18_Weights.DEFAULT

# create preprocessor
preprocessor = weights.transforms()

# load model
model = resnet18(weights=weights)
model = model.eval()

# get class names
class_names = weights.meta['categories']

In [ ]:
# create inverse normalization
renormalizer = transforms.Compose([
    # reverse normalization
    transforms.Normalize(
        mean=[-m/s for m, s in zip(preprocessor.mean, preprocessor.std)],
        std=[1/s for s in preprocessor.std]
    ),
    # clip to valid range
    transforms.Lambda(lambda x: x.clamp(0, 1))
])

## Load image

In [ ]:
# load image
image_path = '../test.jpg'

if not Path(image_path).exists():
    image_path = download_file(
        url='https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg',
        save_path=image_path
    )

image = Image.open(image_path)

In [ ]:
# show image
fig, ax = plt.subplots(figsize=(6, 4))
ax.imshow(np.asarray(image))
ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

## Run model

In [ ]:
# preprocess image
x = preprocessor(image) # (3, h, w)
x = x.unsqueeze(0) # (1, 3, h, w)

# run model
with torch.no_grad():
    logits = model(x) # (1, 1000)

# get predictions
# label_ids = logits.argmax(dim=1) # (1,)
probs = logits.softmax(dim=1) # (1, 1000)
label_probs, label_ids = probs.max(dim=1) # (1,)
labels = [class_names[lidx.item()] for lidx in label_ids]

for l, p in zip(labels, label_probs):
    print(f'Predicted: {l} ({p:.2f})')

## Untargeted FGSM attack

In [ ]:
# perform FGSM attack
fgsm = FGSMAttack(
    model=model,
    criterion=nn.CrossEntropyLoss()
)

fgsm_x = fgsm(
    image=x,
    label=label_ids,
    eps=0.005,
    targeted=False
)

In [ ]:
# run model
with torch.no_grad():
    fgsm_logits = model(fgsm_x) # (1, 1000)

# get predictions
fgsm_probs = fgsm_logits.softmax(dim=1) # (1, 1000)
fgsm_label_probs, fgsm_label_ids = fgsm_probs.max(dim=1) # (1,)
fgsm_labels = [class_names[lidx.item()] for lidx in fgsm_label_ids]

for l, p in zip(fgsm_labels, fgsm_label_probs):
    print(f'Predicted: {l} ({p:.2f})')

In [ ]:
# show images
plot_idx = 0

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ax1.imshow(renormalizer(x[plot_idx]).permute(1, 2, 0).numpy())
ax1.set_aspect('equal', adjustable='box')
ax1.set_title(f'Predicted: {labels[plot_idx]} ({label_probs[plot_idx]:.2f})')

ax2.imshow(renormalizer(fgsm_x[plot_idx]).permute(1, 2, 0).numpy())
ax2.set_aspect('equal', adjustable='box')
ax2.set_title(f'Predicted: {fgsm_labels[plot_idx]} ({fgsm_label_probs[plot_idx]:.2f})')

fig.tight_layout()

## Targeted PGD attack

In [ ]:
# set target label
target_label = 1

print(f'Target: {class_names[target_label]}')

In [ ]:
# perform PGD attack
pgd = PGDAttack(
    model=model,
    criterion=nn.CrossEntropyLoss()
)

pgd_x = pgd(
    image=x,
    label=target_label,
    num_steps=20,
    step_size=0.1,
    eps=5.,
    p_norm=2,
    targeted=True
)

In [ ]:
# run model
with torch.no_grad():
    pgd_logits = model(pgd_x) # (1, 1000)

# get predictions
pgd_probs = pgd_logits.softmax(dim=1) # (1, 1000)
pgd_label_probs, pgd_label_ids = pgd_probs.max(dim=1) # (1,)
pgd_labels = [class_names[lidx.item()] for lidx in pgd_label_ids]

for l, p in zip(pgd_labels, pgd_label_probs):
    print(f'Predicted: {l} ({p:.2f})')

In [ ]:
# show images
plot_idx = 0

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ax1.imshow(renormalizer(x[plot_idx]).permute(1, 2, 0).numpy())
ax1.set_aspect('equal', adjustable='box')
ax1.set_title(f'Predicted: {labels[plot_idx]} ({label_probs[plot_idx]:.2f})')

ax2.imshow(renormalizer(pgd_x[plot_idx]).permute(1, 2, 0).numpy())
ax2.set_aspect('equal', adjustable='box')
ax2.set_title(f'Predicted: {pgd_labels[plot_idx]} ({pgd_label_probs[plot_idx]:.2f})')

fig.tight_layout()